<a href="https://colab.research.google.com/github/SingTown/openmv_tensorflow_training_scripts/blob/main/mobilenet_v2/openmv_mobilenet_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train MobilenetV2 and Save to OpenMV

This Code is for TensorFlow 2 and Keras 3

## Download Cats vs Dogs dataset from github


In [1]:
!git clone --branch master --depth=1 https://github.com/haritha91/Cats-Dogs-Classifier---Keras.git cat_dog

fatal: destination path 'cat_dog' already exists and is not an empty directory.


In [2]:
import tensorflow as tf
import numpy as np
import keras
import os
from keras import layers
from tensorflow import data as tf_data
from tensorflow import lite

CLASS_NUM = 2 # dog and cat
INPUT_SIZE = 96 #image size is 96 * 96 *3
LABELS = ["cats", "dogs"]

# Clean image

In [3]:
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff']
def image_clean(directory):
    for dirpath, _, filenames in os.walk(directory):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            _, ext = os.path.splitext(filename)
            if ext.lower() not in image_extensions:
                os.remove(file_path)
                print(f"Deleted: {file_path}")

image_clean('cat_dog/dataset/training_set')
image_clean('cat_dog/dataset/test_set')

## Load dataset

In [4]:
data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
]

def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

def load_training_data():
    train_ds, val_ds = keras.utils.image_dataset_from_directory(
        "cat_dog/dataset/training_set",
        validation_split=0.2,
        subset="both",
        seed=1337,
        shuffle=True,
        image_size=(INPUT_SIZE, INPUT_SIZE),
        batch_size=32,
        label_mode="categorical",
        class_names=LABELS,
    )
    train_ds = train_ds.map(
        lambda img, label: (data_augmentation(img)/127.5 - 1, label),
        num_parallel_calls=tf_data.AUTOTUNE,
    )
    val_ds = val_ds.map(
        lambda img, label: (img/127.5 - 1, label),
        num_parallel_calls=tf_data.AUTOTUNE,
    )

    # Prefetching samples in GPU memory helps maximize GPU utilization.
    train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
    val_ds = val_ds.prefetch(tf_data.AUTOTUNE)
    return train_ds, val_ds

## Define Model

alpha=0.35 for lower size

In [5]:
def build_model():
    input_shape = (INPUT_SIZE, INPUT_SIZE, 3)
    inputs = layers.Input(shape=input_shape)

    model = keras.applications.MobileNetV2(
        alpha=0.35,
        include_top=False,
        input_shape=input_shape,
        input_tensor=inputs,
    )
    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    x = layers.Dropout(0.1, name="top_dropout")(x)
    outputs = layers.Dense(len(LABELS), activation="softmax", name="pred")(x)

    # Compile
    model = keras.Model(inputs, outputs, name="MobileNetV2")
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

## Train model
After 3 epoches, accuracy is about 91%

In [6]:
train_ds, val_ds = load_training_data()
model = build_model()
model.fit(
    train_ds,
    epochs=3,
    validation_data=val_ds,
)

Found 8000 files belonging to 2 classes.
Using 6400 files for training.
Using 1600 files for validation.
2019640/2019640 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step
Epoch 1/3
200/200 ━━━━━━━━━━━━━━━━━━━━ 37s 103ms/step - accuracy: 0.7843 - loss: 0.5114 - val_accuracy: 0.8975 - val_loss: 0.2496
Epoch 2/3
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 87ms/step - accuracy: 0.8538 - loss: 0.3372 - val_accuracy: 0.9056 - val_loss: 0.2421
Epoch 3/3
200/200 ━━━━━━━━━━━━━━━━━━━━ 17s 85ms/step - accuracy: 0.8539 - loss: 0.3435 - val_accuracy: 0.9087 - val_loss: 0.2424


## Full Integer Quantization

ref: https://www.tensorflow.org/lite/performance/post_training_quantization#full_integer_quantization

In [7]:
def representative_dataset():
    for label in LABELS:
        label_folder = os.path.join('cat_dog/dataset/test_set', label)
        for file_name in os.listdir(label_folder):
            file_path = os.path.join(label_folder, file_name)
            img = keras.utils.load_img(file_path, target_size=(INPUT_SIZE, INPUT_SIZE))
            array = keras.utils.img_to_array(img)
            yield [np.array([array/127.5-1])]

# Convert the tflite.
converter = lite.TFLiteConverter.from_keras_model(model)
converter._experimental_disable_per_channel_quantization_for_dense_layers = True
converter.optimizations = [lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_quant_model = converter.convert()

# Save the model.
with open('trained.tflite', 'wb') as f:
    f.write(tflite_quant_model)

Saved artifact at '/tmp/tmp897vb8iv'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  140007511514896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140007511513552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140007511515088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140007511515472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140007511516432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140007511515664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140007511517392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140007511515856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140007511517584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140007511517200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140007511519696:

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


## Succeed

Copy trained.tflite to OpenMV4 H7 Plus, run this code in OpenMV4 H7 Plus: https://github.com/SingTown/openmv_tensorflow_training_scripts/blob/main/mobilenet_v2/main.py

In [8]:
from google.colab import files
files.download('trained.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>